## Question 1

### (a) What condition(s) must be met for the optimal objective function value to be positive?

#### For the optimal objective function value to be positive, the total revenue from distributing brakes must exceed the total fixed cost of establishing the distribution centers. This requires that at least one DC is opened and that the volume of brakes shipped from the open facility or facilities—when multiplied by the corresponding per-unit revenues—is high enough to cover the fixed cost of operating those centers. Essentially, there must exist a feasible solution where the revenue generated from the shipments outweighs the fixed investment required to set up the network, ensuring a net profit greater than zero.

### (b) Interpret the constraint that links the decision to open a distribution center with their distribution plan, given that the model addresses a capacity planning problem over the next 10 years.

#### The linking constraint, formulated as x[i, j] ≤ 0.45 * Demand[j] * y[i], serves a dual purpose. First, it ensures that if a distribution center is not opened (i.e., y[i] = 0), then no brakes can be distributed from that facility (forcing x[i, j] to zero for all regions j). Second, it limits the volume that can be shipped from an open facility to any region to at most 45% of that region’s demand. This linkage is critical in a 10-year capacity planning problem because it ties the irreversible investment decision of opening a facility to its operational use. Essentially, it guarantees that only facilities that are built can contribute to the distribution plan, and even then, they must operate within realistic bounds that reflect regional demand constraints.

### (c) Benchmarking is useful for providing context. Therefore using Gurobi and assuming constraints 1−8 above are omitted, what would the optimal profit be?

In [1]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data files
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists for facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, and fixed costs
# Assuming capacity is the annual capacity
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build a revenue dictionary:
# Revenue columns in df_cost are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'.
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# Create a new Gurobi model for the benchmark problem (omitting constraints 1-8)
model = Model("Benchmark_10Year")
model.setParam('OutputFlag', 1)  # Enable solver output

# Decision Variables:
# y[i] is a binary variable indicating whether facility i is opened.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] is a continuous variable representing the annual shipments from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# Basic Constraints:

# 1. Capacity Constraint: The total annual shipments from facility i cannot exceed its annual capacity,
#    and shipments are only allowed if the facility is open.
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# 2. Demand Constraint: The total annual shipments to region j cannot exceed its annual demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# Objective Function:
# Calculate the 10-year total revenue (annual revenue multiplied by 10) minus the fixed cost (incurred once).
model.setObjective(
    10 * quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# Optimize the model
model.optimize()

# Print the optimal objective value (10-year profit)
print("Optimal 10-Year Profit (Benchmark):", model.objVal)


Set parameter Username
Set parameter LicenseID to value 2609998
Academic license - for non-commercial use only - expires 2026-01-14
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 544 columns and 1071 nonzeros
Model fingerprint: 0x614e2a46
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 48 rows, 544 columns, 1071 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)

Root relaxation: objective 4.974484e+08, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl

#### The optimal profit is 483175185.0

### (d) After solving the full MILP model with Gurobi, what is the optimal profit?

In [2]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data files
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists for facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16 (0-indexed)
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, fixed costs
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build revenue dictionary:
# Revenue columns in df_cost are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# Create a new Gurobi model for the full MILP model (10-year profit)
model = Model("FullMILP_10Year")
model.setParam('OutputFlag', 1)  # Enable solver output

# Decision Variables:
# y[i] is a binary variable indicating whether facility i is opened.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] is a continuous variable representing the annual shipments from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# ----------------------------
# Basic Constraints
# ----------------------------

# (1) Capacity Constraint: 
# For each facility i, the total annual shipments cannot exceed its capacity if the facility is open.
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# (2) Demand Constraint: 
# For each region j, the total annual shipments received cannot exceed its annual demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# ----------------------------
# Strategic & Operational Constraints (Full MILP)
# ----------------------------

# Note: The problem statement provides locations in 1-indexed form.
# We convert them to 0-indexed based on the CSV files (Facility 0 corresponds to location 1, etc.).

# Constraint 1: At most 3 DCs can be opened at locations {1,2,3,7,11,13,17}.
# 1-indexed locations -> 0-indexed: {0, 1, 2, 6, 10, 12, 16}.
A = [0, 1, 2, 6, 10, 12, 16]
model.addConstr(quicksum(y[i] for i in A) <= 3, name="Constraint1")

# Constraint 2: If a DC is established at location 4 then facilities must also be opened at locations 6,8,10.
# 1-indexed: location 4 -> index 3; locations 6,8,10 -> indices 5,7,9.
model.addConstr(y[3] <= y[5], name="Constraint2a")
model.addConstr(y[3] <= y[7], name="Constraint2b")
model.addConstr(y[3] <= y[9], name="Constraint2c")

# Constraint 3: If a DC is opened at location 2 then at least two DCs must be opened at locations {12,14,16}.
# 1-indexed: location 2 -> index 1; locations 12,14,16 -> indices 11,13,15.
model.addConstr(y[11] + y[13] + y[15] >= 2 * y[1], name="Constraint3")

# Constraint 4: At most 2 DCs can be opened at locations {1,8,9,17}.
# 1-indexed: {1,8,9,17} -> 0-indexed: {0,7,8,16}.
model.addConstr(quicksum(y[i] for i in [0, 7, 8, 16]) <= 2, name="Constraint4")

# Constraint 5: If a DC is opened at location 1 then a facility must be established at location 5 or 6.
# 1-indexed: location 1 -> index 0; locations 5 and 6 -> indices 4 and 5.
model.addConstr(y[0] <= y[4] + y[5], name="Constraint5")

# Constraint 6: The number of DCs opened at locations 1 through 9 must not exceed 1.2 times the number of DCs
# opened at locations 10 through 17.
# 1-indexed: locations 1-9 -> indices 0 to 8; locations 10-17 -> indices 9 to 16.
model.addConstr(quicksum(y[i] for i in range(0, 9)) <= 1.2 * quicksum(y[i] for i in range(9, 17)), name="Constraint6")

# Constraint 7: The total annual shipments from the first 6 DCs (locations 1-6) must be at least 39% of the total annual shipments.
# 1-indexed: locations 1-6 -> indices 0 to 5.
model.addConstr(
    quicksum(x[i, j] for i in range(0, 6) for j in regions) >= 0.39 * quicksum(x[i, j] for i in facilities for j in regions),
    name="Constraint7"
)

# Constraint 8: Linking Constraint – shipments from facility i to region j are allowed only if facility i is open,
# and each shipment cannot exceed 45% of the region's annual demand.
model.addConstrs(
    (x[i, j] <= 0.45 * demand[j] * y[i] for i in facilities for j in regions),
    name="Constraint8"
)

# ----------------------------
# Objective Function
# ----------------------------
# The model maximizes the 10-year profit.
# Annual revenue is multiplied by 10, while fixed costs are incurred only once.
model.setObjective(
    10 * quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# Optimize the model
model.optimize()

# Print the optimal objective value (10-year profit)
print("Optimal 10-Year Profit (Full MILP):", model.objVal)


Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 584 rows, 544 columns and 2693 nonzeros
Model fingerprint: 0xc82b3982
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 584 rows, 544 columns, 2693 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)

Root relaxation: objective 4.392851e+08, 183 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.3929e+08    0    7   -0.00000 4.3929e+08      - 

### (e) Discuss the value of benchmarking by comparing the optimal objective function values from the previous two questions. What managerial insights can be drawn from this comparison?

#### Benchmarking offers a valuable reference point by showing the maximum potential profit without practical constraints. In our model, when all strategic and operational restrictions are omitted, the optimal 10-year profit is approximately \$483 million. However, once all constraints are included, the profit decreases to about \$424 million. This roughly \$59 million gap highlights the trade-offs that come with implementing real-world restrictions—such as location limits, dependency requirements, and shipment caps. For managers, this comparison emphasizes that while a theoretical model might suggest very high profitability, the necessary operational constraints significantly lower this figure. Understanding the cost imposed by these constraints can guide decision-makers in evaluating whether some restrictions might be adjusted or if investments should be made to mitigate their impact, ultimately helping to balance profitability with practical considerations.

### (f) If we solved the linear relaxation of this problem, it would be classified as a linear program. Although this would simplify the problem, the resulting optimal solution would not be integral. Explain why using this method to solve the problem faster is not practical in this case.

#### Solving the linear relaxation would indeed simplify the computation by allowing fractional values for the decision variables, but this approach is not practical in our case because the model's key decisions are inherently discrete. In our MILP, variables like \(y_i\) represent the decision to open a facility, and they must be binary—either a facility is opened (1) or not (0). When relaxed, these values could fall between 0 and 1 (for example, 0.6), which does not have any real-world interpretation (a facility cannot be "partially" open). Moreover, rounding these fractional solutions to obtain an integer solution could lead to infeasibilities or suboptimal outcomes due to the tight coupling between fixed costs, capacities, and shipping decisions. In essence, although LP relaxation provides a computationally faster bound on the objective function, the resulting solution would not be directly implementable for strategic capacity planning over the 10-year period, making the use of full MILP methods necessary to capture the discrete, combinatorial nature of the problem.

### (g) For the full MILP model, how many feasible solutions are within 1% of the optimal solution? What is the smallest objective function value of the feasible solution(s) you have found?

In [3]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# ----------------------------
# Data Loading and Setup
# ----------------------------
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists of facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16 (0-indexed)
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, and fixed costs
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build a revenue dictionary
# Revenue columns are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# ----------------------------
# Create Model and Set Parameters for Solution Pool
# ----------------------------
model = Model("FullMILP_10Year_SolutionPool")
model.setParam('OutputFlag', 1)       # Enable solver output
model.setParam('PoolSearchMode', 2)     # Enable solution pool search
model.setParam('PoolSolutions', 1000)    # Request up to 1000 solutions

# ----------------------------
# Decision Variables
# ----------------------------
# y[i] = 1 if facility i is open, 0 otherwise.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] represents the annual shipments from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# ----------------------------
# Basic Constraints
# ----------------------------
# (1) Capacity Constraint:
# For each facility i, the total annual shipments must not exceed its capacity if the facility is open.
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# (2) Demand Constraint:
# For each region j, the total annual shipments received must not exceed the region's annual demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# ----------------------------
# Strategic & Operational Constraints (Converted to 0-indexed)
# ----------------------------
# Constraint 1: At most 3 DCs can be opened at locations {1,2,3,7,11,13,17} (1-indexed -> indices {0,1,2,6,10,12,16})
A = [0, 1, 2, 6, 10, 12, 16]
model.addConstr(quicksum(y[i] for i in A) <= 3, name="Constraint1")

# Constraint 2: If a DC is established at location 4 (index 3), then facilities must also be opened at locations 6,8,10 (indices 5,7,9)
model.addConstr(y[3] <= y[5], name="Constraint2a")
model.addConstr(y[3] <= y[7], name="Constraint2b")
model.addConstr(y[3] <= y[9], name="Constraint2c")

# Constraint 3: If a DC is opened at location 2 (index 1), then at least two DCs must be opened at locations {12,14,16} (indices 11,13,15)
model.addConstr(y[11] + y[13] + y[15] >= 2 * y[1], name="Constraint3")

# Constraint 4: At most 2 DCs can be opened at locations {1,8,9,17} (1-indexed -> indices {0,7,8,16})
model.addConstr(quicksum(y[i] for i in [0, 7, 8, 16]) <= 2, name="Constraint4")

# Constraint 5: If a DC is opened at location 1 (index 0), then a facility must be established at location 5 or 6 (indices 4,5)
model.addConstr(y[0] <= y[4] + y[5], name="Constraint5")

# Constraint 6: The number of DCs opened at locations 1 through 9 (indices 0-8) must not exceed 1.2 times the number opened at locations 10 through 17 (indices 9-16)
model.addConstr(quicksum(y[i] for i in range(0, 9)) <= 1.2 * quicksum(y[i] for i in range(9, 17)), name="Constraint6")

# Constraint 7: The total annual shipments from the first 6 DCs (locations 1-6, indices 0-5) must be at least 39% of the total annual shipments.
model.addConstr(
    quicksum(x[i, j] for i in range(0, 6) for j in regions) >= 0.39 * quicksum(x[i, j] for i in facilities for j in regions),
    name="Constraint7"
)

# Constraint 8: Linking Constraint – shipments from facility i to region j are only allowed if facility i is open,
# and each shipment cannot exceed 45% of the region's annual demand.
model.addConstrs(
    (x[i, j] <= 0.45 * demand[j] * y[i] for i in facilities for j in regions),
    name="Constraint8"
)

# ----------------------------
# Objective Function
# ----------------------------
# Maximize 10-year profit: 10 times the annual revenue minus the fixed cost (incurred only once)
model.setObjective(
    10 * quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# ----------------------------
# Optimize the Model
# ----------------------------
model.optimize()

# Print the optimal objective value (10-year profit)
best_obj = model.objVal
print("Optimal 10-Year Profit (Full MILP):", best_obj)

# ----------------------------
# Retrieve and Analyze the Solution Pool for Question (g)
# ----------------------------
sol_count = model.SolCount
print("Number of solutions in the pool:", sol_count)

# Count the number of solutions within 1% of the optimal objective and find the smallest objective among them.
within_gap_count = 0
min_obj_within_gap = float('inf')
for i in range(sol_count):
    # Set the solution number to i to query that solution's objective value.
    model.setParam(GRB.Param.SolutionNumber, i)
    sol_obj = model.PoolobjVal  # Get objective value for the current solution
    # For a maximization problem, a solution is within 1% of optimal if its objective value is at least 99% of the best objective.
    if sol_obj >= 0.99 * best_obj:
        within_gap_count += 1
        if sol_obj < min_obj_within_gap:
            min_obj_within_gap = sol_obj

print("Number of solutions within 1% of the optimal solution:", within_gap_count)
print("Smallest objective value among these solutions:", min_obj_within_gap)


Set parameter OutputFlag to value 1
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1000
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  1000
PoolSearchMode  2

Optimize a model with 584 rows, 544 columns and 2693 nonzeros
Model fingerprint: 0xc82b3982
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 584 rows, 544 columns, 2693 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)
Found heuristic solution: objective 3.903807e+08

Root relaxation: objective 4.392851e+08, 183 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Cu

#### There are 3 feasible solutions are within 1% of the optimal solution. And the smallest objective function value of the feasible solution is 419564405.92

### (h) Finding both the optimal solution and several sub-optimal solutions within a relative optimality gap can be challenging for more complex problems. How can you configure Gurobi to return the first feasible solution it finds within a specified percentage of the optimal solution? Using this parameter setting, what is the objective function value for a solution within 5% of optimality?

In [4]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# ----------------------------
# Data Loading and Setup
# ----------------------------
# Load the data files
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists for facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16 (0-indexed)
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, and fixed costs
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build revenue dictionary:
# Revenue columns in df_cost are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# ----------------------------
# Model Setup: Full MILP for 10-Year Profit (Aggregated 2D formulation)
# ----------------------------
model = Model("FullMILP_10Year")
model.setParam('OutputFlag', 1)  # Enable solver output

# Set MIPGap to 0.05 so that the solver stops as soon as it finds a solution within 5% of optimality.
model.setParam('MIPGap', 0.05)


# ----------------------------
# Create Model and Set Parameters for Solution Pool
# ----------------------------
model = Model("FullMILP_10Year_SolutionPool")
model.setParam('OutputFlag', 1)       # Enable solver output
model.setParam('PoolSearchMode', 2)     # Enable solution pool search
model.setParam('PoolSolutions', 1000)    # Request up to 1000 solutions


# Decision Variables:
# y[i] is a binary variable indicating if facility i is opened.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] represents the annual shipments from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# ----------------------------
# Basic Constraints
# ----------------------------
# (1) Capacity Constraint: For each facility, total annual shipments cannot exceed its capacity (if the facility is open).
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# (2) Demand Constraint: For each region, total annual shipments received cannot exceed its demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# ----------------------------
# Strategic & Operational Constraints
# (Converted from 1-indexed to 0-indexed)
# ----------------------------
# Constraint 1: At most 3 DCs can be opened at locations {1,2,3,7,11,13,17} (0-indexed: {0,1,2,6,10,12,16})
A = [0, 1, 2, 6, 10, 12, 16]
model.addConstr(quicksum(y[i] for i in A) <= 3, name="Constraint1")

# Constraint 2: If a DC is established at location 4 then facilities must also be opened at locations 6, 8, 10.
# (0-indexed: location 4 -> index 3; locations 6,8,10 -> indices 5,7,9)
model.addConstr(y[3] <= y[5], name="Constraint2a")
model.addConstr(y[3] <= y[7], name="Constraint2b")
model.addConstr(y[3] <= y[9], name="Constraint2c")

# Constraint 3: If a DC is opened at location 2 then at least two DCs must be opened at locations {12,14,16}.
# (0-indexed: location 2 -> index 1; locations 12,14,16 -> indices 11,13,15)
model.addConstr(y[11] + y[13] + y[15] >= 2 * y[1], name="Constraint3")

# Constraint 4: At most 2 DCs can be opened at locations {1,8,9,17}.
# (0-indexed: {1,8,9,17} -> {0,7,8,16})
model.addConstr(quicksum(y[i] for i in [0, 7, 8, 16]) <= 2, name="Constraint4")

# Constraint 5: If a DC is opened at location 1 then a facility must be established at location 5 or 6.
# (0-indexed: location 1 -> index 0; locations 5 and 6 -> indices 4 and 5)
model.addConstr(y[0] <= y[4] + y[5], name="Constraint5")

# Constraint 6: The number of DCs opened at locations 1 through 9 must not exceed 1.2 times the number of DCs opened at locations 10 through 17.
# (0-indexed: locations 1-9 -> indices 0 to 8; locations 10-17 -> indices 9 to 16)
model.addConstr(quicksum(y[i] for i in range(0, 9)) <= 1.2 * quicksum(y[i] for i in range(9, 17)), name="Constraint6")

# Constraint 7: Total annual shipments from the first 6 DCs must be at least 39% of total annual shipments.
# (0-indexed: locations 1-6 -> indices 0 to 5)
model.addConstr(
    quicksum(x[i, j] for i in range(0, 6) for j in regions) >= 0.39 * quicksum(x[i, j] for i in facilities for j in regions),
    name="Constraint7"
)

# ----------------------------
# Linking Constraint (Constraint 8)
# ----------------------------
# For each facility and region, shipments are allowed only if the facility is open,
# and shipments cannot exceed 45% of the region's annual demand.
model.addConstrs(
    (x[i, j] <= 0.45 * demand[j] * y[i] for i in facilities for j in regions),
    name="Constraint8"
)

# ----------------------------
# Objective Function
# ----------------------------
# Maximize 10-year profit: (10 * annual revenue) - fixed cost (incurred only once)
model.setObjective(
    10 * quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# ----------------------------
# Solve the Model
# ----------------------------
model.optimize()

# ----------------------------
# Output the objective function value of the first solution found within 5% optimality.
# Since we set MIPGap = 0.05, the solver stops as soon as it finds a solution with a relative gap of 5% or less.
#print("Objective function value for solution within 5% of optimality:", model.objVal)


Set parameter OutputFlag to value 1
Set parameter MIPGap to value 0.05
Set parameter OutputFlag to value 1
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1000
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  1000
PoolSearchMode  2

Optimize a model with 584 rows, 544 columns and 2693 nonzeros
Model fingerprint: 0xc82b3982
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 584 rows, 544 columns, 2693 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)
Found heuristic solution: objective 3.903807e+08

Root relaxation: objective 4.392851e+0

#### To configure Gurobi so that it returns the first feasible solution within a specified percentage of the optimal, you can set the MIPGap parameter to that value—in this case, 0.05 (or 5%). This instructs Gurobi to stop the search as soon as it finds a solution whose objective value is within 5% of the best bound. Based on the results with MIPGap set to 0.05, the solver returned a solution with an objective function value of approximately \$418,468,172.54. This solution meets the 5% optimality gap criterion, as the gap reported was about 4.97%, indicating that the returned solution is within 5% of the optimal solution.

### (i) Write down two additional Gurobi parameters that can be used to find feasible but sub-optimal solutions for difficult optimization problems, and briefly explain why they are applicable.


**TimeLimit**

One useful parameter is `TimeLimit`. By setting `TimeLimit` to a specified number of seconds (for example, `TimeLimit = 60`), Gurobi will terminate the optimization after that duration and return the best feasible solution found up to that point. This is particularly helpful when solving difficult problems where finding the absolute optimal solution may be time‐consuming. Mathematically, if $T_{\text{max}}$ is the time limit, then the solver returns a solution with objective value

$$
f^* \approx \max_{t \leq T_{\text{max}}} f(t)
$$

where $f(t)$ is the best objective value found by time $t$.

**MIPFocus**

Another beneficial parameter is `MIPFocus`. For instance, setting `MIPFocus = 1` instructs Gurobi to prioritize finding feasible solutions over aggressively closing the optimality gap. This setting biases the branch-and-bound process toward exploring parts of the search tree that are more likely to yield feasible (even if sub-optimal) solutions quickly. Such a configuration is particularly valuable in complex optimization problems where obtaining a good feasible solution in a reasonable amount of time is more critical than proving global optimality.


### (j) Explain why, in cases like this, using Gurobi parameters to solve an approximate version of an optimization problem is preferable to solving an LP relaxation of the same model.

#### Using Gurobi parameters to obtain an approximate MILP solution (for example, by setting a gap tolerance) provides a feasible integer solution that respects all logical and capacity constraints. In contrast, solving the LP relaxation ignores integrality, allowing fractional values that may not be implementable (such as “opening half a facility”). Even if the LP relaxation provides a tight upper bound on the objective, its solution is not directly usable in practice and must be rounded—potentially introducing infeasibilities or significant suboptimality. By configuring Gurobi to stop at a given optimality gap, you ensure the solver returns a valid, integer solution that is within a known percentage of optimal, offering both realism (integer decisions) and efficiency (controlled solve time).

## Question 2

### (e) Create a stochastic program using a Monte Carlo simulation of 20 trials with 10 scenarios per trial (i.e., use SAA). What is the optimal expected cost?

In [5]:
import pandas as pd
import numpy as np
from gurobipy import *

costs_df = pd.read_csv('/Users/Sam/Downloads/costs.csv', index_col=0)
randomness_df = pd.read_csv('/Users/Sam/Downloads/randomness.csv',index_col=0)

In [6]:

# Travel cost matrix
nodes = list(costs_df.index)  # Station_0 to Station_14
stations = nodes[1:]          # Exclude Station_0 (the depot)

# Demand parameters

# Parameters
num_trials = 20
scenarios_per_trial = 10
penalty_small = 0.13
penalty_big = 0.09
np.random.seed(42)  # For reproducibility

# === Step 2: Helper function to generate scenarios ===
def generate_scenarios():
    scenarios = []
    for _ in range(scenarios_per_trial):
        demand_realized = {}
        for station in stations:
            prob = randomness_df.loc[station, 'Probability']
            if np.random.rand() < prob:
                mu = randomness_df.loc[station, 'Mean_Demand']
                sigma = randomness_df.loc[station, 'Std_Dev_Demand']
                demand = max(0, np.random.normal(mu, sigma))
                demand_realized[station] = demand
        scenarios.append(demand_realized)
    return scenarios

# === Step 3: Solve one SAA trial ===
def solve_trial(scenarios):
    m = Model()
    m.setParam('OutputFlag', 0)

    # Variables
    x = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
    u = m.addVars(stations, vtype=GRB.CONTINUOUS, lb=1, ub=len(stations), name="u")
    C = m.addVar(vtype=GRB.CONTINUOUS, name="TruckCapacity")

    # Travel cost (first-stage)
    travel_cost = quicksum(costs_df.loc[i, j] * x[i, j] for i in nodes for j in nodes if i != j)

    # Second-stage penalties
    penalties = []
    for idx, scen in enumerate(scenarios):
        total_demand = sum(scen[st] for st in scen)

        # Slack variables
        over_amount = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"over_{idx}")
        under_amount = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"under_{idx}")

        # Constraints for penalties
        m.addConstr(over_amount >= C - total_demand)
        m.addConstr(under_amount >= total_demand - C)

        penalties.append(penalty_big * over_amount + penalty_small * under_amount)

    expected_penalty = quicksum(penalties) / scenarios_per_trial
    m.setObjective(travel_cost + expected_penalty, GRB.MINIMIZE)

    # Flow constraints
    for i in nodes:
        if i != 'Station_0':
            m.addConstr(quicksum(x[j, i] for j in nodes if j != i) == 1)
            m.addConstr(quicksum(x[i, j] for j in nodes if j != i) == 1)
    m.addConstr(quicksum(x['Station_0', j] for j in stations) == 1)
    m.addConstr(quicksum(x[i, 'Station_0'] for i in stations) == 1)

    # Subtour elimination (MTZ)
    for i in stations:
        for j in stations:
            if i != j:
                m.addConstr(u[i] - u[j] + len(stations) * x[i, j] <= len(stations) - 1)

    m.optimize()
    return m.objVal

# === Step 4: Run 20 trials ===
trial_costs = []
for t in range(num_trials):
    scenarios = generate_scenarios()
    cost = solve_trial(scenarios)
    print(f"Trial {t+1} Expected Cost: {cost:.2f}")
    trial_costs.append(cost)

# Final report
optimal_expected_cost = np.mean(trial_costs)
print("\n==============================")
print(f"Estimated Optimal Expected Cost: {optimal_expected_cost:.2f}")

Trial 1 Expected Cost: 176.80
Trial 2 Expected Cost: 184.51
Trial 3 Expected Cost: 180.69
Trial 4 Expected Cost: 183.31
Trial 5 Expected Cost: 184.09
Trial 6 Expected Cost: 174.34
Trial 7 Expected Cost: 184.11
Trial 8 Expected Cost: 183.13
Trial 9 Expected Cost: 186.04
Trial 10 Expected Cost: 166.11
Trial 11 Expected Cost: 170.40
Trial 12 Expected Cost: 178.80
Trial 13 Expected Cost: 190.90
Trial 14 Expected Cost: 193.85
Trial 15 Expected Cost: 189.78
Trial 16 Expected Cost: 190.89
Trial 17 Expected Cost: 171.71
Trial 18 Expected Cost: 201.70
Trial 19 Expected Cost: 193.65
Trial 20 Expected Cost: 172.38

Estimated Optimal Expected Cost: 182.86


#### The optimal expected cost is 182.86

### (f) Using a Monte Carlo simulation of 20 trials with 10 scenarios per trial, what is the expected value of reacting with perfect foresight (WS) and the expected value of perfect information (EVPI) associated with this stochastic program?

In [7]:
# === Step 5: Compute WS and EVPI ===
def solve_perfect_information(scen):
    m = Model()
    m.setParam('OutputFlag', 0)

    # Variables
    x = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
    u = m.addVars(stations, vtype=GRB.CONTINUOUS, lb=1, ub=len(stations), name="u")
    C = m.addVar(vtype=GRB.CONTINUOUS, name="TruckCapacity")

    # Total demand is known
    total_demand = sum(scen[st] for st in scen)

    # Slack variables
    over_amount = m.addVar(lb=0, vtype=GRB.CONTINUOUS)
    under_amount = m.addVar(lb=0, vtype=GRB.CONTINUOUS)

    m.addConstr(over_amount >= C - total_demand)
    m.addConstr(under_amount >= total_demand - C)

    # Objective
    travel_cost = quicksum(costs_df.loc[i, j] * x[i, j] for i in nodes for j in nodes if i != j)
    penalty = penalty_big * over_amount + penalty_small * under_amount
    m.setObjective(travel_cost + penalty, GRB.MINIMIZE)

    # Flow constraints
    for i in nodes:
        if i != 'Station_0':
            m.addConstr(quicksum(x[j, i] for j in nodes if j != i) == 1)
            m.addConstr(quicksum(x[i, j] for j in nodes if j != i) == 1)
    m.addConstr(quicksum(x['Station_0', j] for j in stations) == 1)
    m.addConstr(quicksum(x[i, 'Station_0'] for i in stations) == 1)

    # MTZ subtour elimination
    for i in stations:
        for j in stations:
            if i != j:
                m.addConstr(u[i] - u[j] + len(stations) * x[i, j] <= len(stations) - 1)

    m.optimize()
    return m.objVal

# Generate same scenarios
all_scenarios = []
for _ in range(num_trials):
    all_scenarios.extend(generate_scenarios())  # 20 * 10 = 200 scenarios

# Solve each with perfect info
ws_costs = []
for idx, scen in enumerate(all_scenarios):
    cost = solve_perfect_information(scen)
    print(f"WS Scenario {idx+1} Cost: {cost:.2f}")
    ws_costs.append(cost)

WS = np.mean(ws_costs)
EVPI = optimal_expected_cost - WS

print("\n==============================")
print(f"Wait-and-See (WS) Value: {WS:.2f}")
print(f"Expected Value of Perfect Information (EVPI): {EVPI:.2f}")

WS Scenario 1 Cost: 142.00
WS Scenario 2 Cost: 142.00
WS Scenario 3 Cost: 142.00
WS Scenario 4 Cost: 142.00
WS Scenario 5 Cost: 142.00
WS Scenario 6 Cost: 142.00
WS Scenario 7 Cost: 142.00
WS Scenario 8 Cost: 142.00
WS Scenario 9 Cost: 142.00
WS Scenario 10 Cost: 142.00
WS Scenario 11 Cost: 142.00
WS Scenario 12 Cost: 142.00
WS Scenario 13 Cost: 142.00
WS Scenario 14 Cost: 142.00
WS Scenario 15 Cost: 142.00
WS Scenario 16 Cost: 142.00
WS Scenario 17 Cost: 142.00
WS Scenario 18 Cost: 142.00
WS Scenario 19 Cost: 142.00
WS Scenario 20 Cost: 142.00
WS Scenario 21 Cost: 142.00
WS Scenario 22 Cost: 142.00
WS Scenario 23 Cost: 142.00
WS Scenario 24 Cost: 142.00
WS Scenario 25 Cost: 142.00
WS Scenario 26 Cost: 142.00
WS Scenario 27 Cost: 142.00
WS Scenario 28 Cost: 142.00
WS Scenario 29 Cost: 142.00
WS Scenario 30 Cost: 142.00
WS Scenario 31 Cost: 142.00
WS Scenario 32 Cost: 142.00
WS Scenario 33 Cost: 142.00
WS Scenario 34 Cost: 142.00
WS Scenario 35 Cost: 142.00
WS Scenario 36 Cost: 142.00
W

#### The expected value of WS expected cost is 142.00 and the EVPI expected value of perfect information is 40.86

### (g) Using a Monte Carlo simulation of 20 trials with 10 scenarios per trial, what is the expected value of using the solution from the mean value problem (EEV) and the value of the stochastic solution (VSS) associated with this stochastic program? To compute the EV solution, assume that the fuel truck visits every customer and satisfies their mean demand.

In [8]:
# === Step 6: Solve Expected Value problem (EV solution) ===
def solve_EV_solution():
    m = Model()
    m.setParam('OutputFlag', 0)

    # Use all stations (since EV assumes every customer needs fuel)
    x = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
    u = m.addVars(stations, vtype=GRB.CONTINUOUS, lb=1, ub=len(stations), name="u")
    C = m.addVar(vtype=GRB.CONTINUOUS, name="TruckCapacity")

    # Total mean demand
    total_mean_demand = randomness_df['Mean_Demand'].sum()

    # Slack variables
    over = m.addVar(lb=0, vtype=GRB.CONTINUOUS)
    under = m.addVar(lb=0, vtype=GRB.CONTINUOUS)
    m.addConstr(over >= C - total_mean_demand)
    m.addConstr(under >= total_mean_demand - C)

    travel_cost = quicksum(costs_df.loc[i, j] * x[i, j] for i in nodes for j in nodes if i != j)
    penalty = penalty_big * over + penalty_small * under
    m.setObjective(travel_cost + penalty, GRB.MINIMIZE)

    # Route constraints
    for i in nodes:
        if i != 'Station_0':
            m.addConstr(quicksum(x[j, i] for j in nodes if j != i) == 1)
            m.addConstr(quicksum(x[i, j] for j in nodes if j != i) == 1)
    m.addConstr(quicksum(x['Station_0', j] for j in stations) == 1)
    m.addConstr(quicksum(x[i, 'Station_0'] for i in stations) == 1)

    for i in stations:
        for j in stations:
            if i != j:
                m.addConstr(u[i] - u[j] + len(stations) * x[i, j] <= len(stations) - 1)

    m.optimize()

    route = {(i, j): x[i, j].X for i in nodes for j in nodes if i != j and x[i, j].X > 0.5}
    capacity = C.X
    return route, capacity

# === Step 7: Evaluate EEV (Expected cost using EV solution) ===
def evaluate_EEV(route, capacity, scenarios):
    costs = []
    for scen in scenarios:
        m = Model()
        m.setParam('OutputFlag', 0)

        # Fixed capacity
        C = m.addVar(lb=0, vtype=GRB.CONTINUOUS)
        m.addConstr(C == capacity)

        # Fixed travel cost
        travel_cost = sum(costs_df.loc[i, j] for (i, j) in route)

        total_demand = sum(scen[st] for st in scen)
        over = m.addVar(lb=0, vtype=GRB.CONTINUOUS)
        under = m.addVar(lb=0, vtype=GRB.CONTINUOUS)
        m.addConstr(over >= C - total_demand)
        m.addConstr(under >= total_demand - C)

        penalty = penalty_big * over + penalty_small * under
        m.setObjective(travel_cost + penalty, GRB.MINIMIZE)
        m.optimize()
        costs.append(m.objVal)

    return np.mean(costs)

# Step 6: Solve EV
ev_route, ev_capacity = solve_EV_solution()

# Step 7: Evaluate EEV
eev_cost = evaluate_EEV(ev_route, ev_capacity, all_scenarios)

# Step 8: VSS
VSS = eev_cost - optimal_expected_cost

print("\n==============================")
print(f"EEV (Expected cost using EV solution): {eev_cost:.2f}")
print(f"VSS (Value of Stochastic Solution): {VSS:.2f}")


EEV (Expected cost using EV solution): 287.41
VSS (Value of Stochastic Solution): 104.55


#### The expected value of EEV expected value is 287.41 and the VSS expected value is 104.55